In [ ]:
# install requirements
!pip install --upgrade pip -q
!pip install --upgrade transformers
!pip install huggingface_hub
!pip install datasets~=2.16.1
!pip install openai~=1.11.0
!pip install scikit-learn
!pip install pandas
!pip install tqdm
!pip install python-dotenv
!pip install aiohttp
!pip install certifi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 162.9 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.42.4
    Uninstalling transformers-4.42.4:
      Successfully uninstalled transformers-4.42.4
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.6.1
    Uninstalling fsspec-2024.6.1:
      Successfully uninstalled fsspec-2024.6.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.6.1 requires fsspec==2024.6.1, but you have fsspec 2023.10.0 which is incompatible.


In [ ]:

import os
from tqdm import tqdm
import ast
import numpy as np
import pandas as pd
import random
import json
import requests
from datetime import datetime

from datasets import load_dataset
from datasets import concatenate_datasets
from openai import OpenAI, AsyncOpenAI
import openai

import asyncio
from aiohttp import ClientSession, ClientTimeout, ClientError
from tqdm import tqdm
import random
import logging

print("Notebook running")

Notebook running


In [ ]:
# login via the huggingface hub with you hf_token
# you need a huggingface account and create a token here: https://huggingface.co/settings/tokens
# we can then call on the token with huggingface_hub.get_token()
import huggingface_hub
huggingface_hub.login()

In [ ]:
# global variables for APIs
MODEL = "Mixtral-8x7B-Instruct-v0.1"
API_URL = f"https://z6iaklmgr9hwh0mq.us-east-1.aws.endpoints.huggingface.cloud"
HEADERS = {"Accept" : "application/json","Authorization": f"Bearer {huggingface_hub.get_token()}",
	"Content-Type": "application/json" }

API_PROVIDER = "HF"

# for asynchronous API calls
BATCH_SIZE = 50 #### needs to be changed
SLEEP_TIME = 3

# global variables for experiment variations
SEED = 333
SELF_CONSISTENCY_ITERATIONS = 3  # How many times should the model try to predict the same text for self-consistency?

In [ ]:
random.seed(SEED)

dataset = pd.read_csv('surrogate_sample_prop.csv')

print(dataset)

           date                                           sentence      label  \
0    19.02.2015  Evidence from the January 2015 bank lending su...    certain   
1    19.02.2015  Inflation outturns had been on a continuous do...    certain   
2    19.02.2015  It could therefore be considered that there wa...  uncertain   
3    19.02.2015  On the basis of its reassessment of the outloo...    certain   
4    19.02.2015  The incoming information had been broadly in l...    certain   
..          ...                                                ...        ...   
495  10.05.2024  With regard to price developments, members con...    certain   
496  10.05.2024  Concerns were expressed that the labour market...  uncertain   
497  19.02.2015  As a further complement to the expanded asset ...    certain   
498  19.02.2015  Moreover, in the United States the capital mar...  uncertain   
499  19.02.2015  This would comprise the purchase of public sec...    certain   

     sampling_prob  row_num

In [ ]:
# Prompt

prompt_ecb_uncertainty_cot = """
You are a highly qualified expert trained to annotate machine learning training data.

Your task is to analyze the uncertainty in the TEXT below from an economic researcher or investor perspective and label it with only one of the two labels: uncertain or certain.

Label the TEXT as uncertain if it includes any indication of doubt, ambiguity, or speculation, such as the use of words or phrases like "could," "might," "should," "possibly," "perhaps," "seem," "likely," "unlikely," "appears," "unclear," "uncertain,"
or hedging language like "it is believed that" or "expected to." Additionally, label a sentence as uncertain if it presents mixed signals, conflicting indicators, or any suggestions where outcomes are not definitively stated. A sentence should also be
labeled uncertain if it reflects on potential scenarios or outcomes that depend on multiple variables, future events, or missing information. However, even with uncertainty-related words, if the sentence presents a clear factual base, strong consensus,
or a well-established truth that remains uncontested, it should be examined closely to ensure the overall context justifies an uncertain label.

Label the TEXT as certain if it expresses a clear, definitive conclusion, or presents information without any indication of doubt. Sentences should be labeled certain if they describe actions, events, or decisions that are presented as factual, concrete,
or based on widely accepted knowledge. Even if a sentence discusses future events, if it is rooted in strong factual statements, well-supported projections, or universal truths, and does not leave room for ambiguity, it should be considered certain.
Additionally, sentences that outline specific outcomes, results, or actions that are not contingent on unknown variables, or that reflect a consensus or widely acknowledged view, should be labeled as certain.

Consider carefully whether any hedging or speculative language is actually present before labeling as uncertain. Be attentive to the difference between discussing multiple possibilities (uncertain) and presenting a well-grounded interpretation of current data or trends (certain).

Base your label decision only on the TEXT and avoid speculation beyond what is explicitly stated.

You first reason step by step about the correct label and then return your label.

You ALWAYS respond only in the following JSON format: {{"reason": "...", "label": "..."}}
You only respond with one single JSON response.

Examples:
Text: Looking ahead, survey-based measures of inflation expectations had declined.
JSON response: {{"reason": "The text states a definitive and observed fact about the decline in survey-based measures of inflation expectations", "label": "certain"}}
Text: Ms Schnabel reviewed the latest financial market developments.
JSON response: {{"reason": "The text simply describes an action without providing any evaluative information or indicating the outcome of the review", "label": "certain"}}
Text: The latest indicators on trade gave mixed signals while pointing overall towards broad stabilisation.
JSON response: {{"reason": "The text acknowledges mixed signals from trade indicators, indicating ambiguity and a lack of definitive conclusion", "label": "uncertain"}}
Text: Moreover, it remained to be seen by how much indirect effects would contribute to a further rise in core inflation over time.
JSON response: {{"reason": "The text mentions that contribution of indirect effects on core inflation are yet to be seen and are unknown, indicating uncertainty and a lack of definitive conclusion", "label": "uncertain"}}

Your TEXT to analyse:
TEXT: {text}
JSON response: """

In [ ]:
from transformers import AutoTokenizer
model_id = "mistralai/Mixtral-8x7B-Instruct-v0.1"
tokenizer = AutoTokenizer.from_pretrained(model_id)

# chat_ecb_uncertainty = [{"role": "user", "content": prompt_ecb_uncertainty}]
chat_ecb_uncertainty_cot = [{"role": "user", "content": prompt_ecb_uncertainty_cot}]

# prompt_ecb_uncertainty_formatted = tokenizer.apply_chat_template(chat_ecb_uncertainty, tokenize=False)
prompt_ecb_uncertainty_cot_formatted = tokenizer.apply_chat_template(chat_ecb_uncertainty_cot, tokenize=False)

prompt_ecb_uncertainty_cot_formatted

'<s> [INST] \nYou are a highly qualified expert trained to annotate machine learning training data.\n\nYour task is to analyze the uncertainty in the TEXT below from an economic researcher or investor perspective and label it with only one of the two labels: uncertain or certain.\n\nLabel the TEXT as uncertain if it includes any indication of doubt, ambiguity, or speculation, such as the use of words or phrases like "could," "might," "should," "possibly," "perhaps," "seem," "likely," "unlikely," "appears," "unclear," "uncertain,"\nor hedging language like "it is believed that" or "expected to." Additionally, label a sentence as uncertain if it presents mixed signals, conflicting indicators, or any suggestions where outcomes are not definitively stated. A sentence should also be\nlabeled uncertain if it reflects on potential scenarios or outcomes that depend on multiple variables, future events, or missing information. However, even with uncertainty-related words, if the sentence presen

In [ ]:
# check token counts

# Encode the formatted prompts to get the token IDs
tokens_ecb_uncertainty_cot = tokenizer.encode(prompt_ecb_uncertainty_cot_formatted, return_tensors='pt')

# Get the number of tokens
num_tokens_ecb_uncertainty_cot = tokens_ecb_uncertainty_cot.size(1)

# Print the number of tokens
print(f"Number of tokens in the ECB Uncertainty CoT query: {num_tokens_ecb_uncertainty_cot}")

Number of tokens in the ECB Uncertainty CoT query: 801


In [ ]:
# params for API: https://huggingface.co/docs/api-inference/detailed_parameters#text-generation-task
# alternative list for API: https://huggingface.github.io/text-generation-inference/#/Text%20Generation%20Inference/generate
# params for endpoints: https://huggingface.co/docs/huggingface_hub/v0.20.3/en/package_reference/inference_client#huggingface_hub.InferenceClient

generation_params = dict(
    top_p=0.90,
    top_k=None,
    temperature=0.4,
    repetition_penalty=1.0,
    do_sample=True,
    max_new_tokens=512,
    return_full_text=False,
    #seed=SEED,  # no seed, because we need randomness for self-consistency
    max_time=None,
    stream=False,
    details=False,
    use_cache=False,
    wait_for_model=False,
)


In [ ]:
# avoid error: Request failed due to network error: Cannot connect to host api-inference.huggingface.co:443 ssl:True
# [SSLCertVerificationError: (1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1006)')]
# this might not be necessary on your machine
import ssl
import certifi
import aiohttp
ssl_context = ssl.create_default_context(cafile=certifi.where())

In [ ]:
# asynchronous functions for efficiently calling on LLM APIs with batching
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')


# functions for calling the HF API with retries, async and batch processing
async def request_with_retry_hf(session, url, headers, json, semaphore, retries=4, backoff_factor=3):
    """Attempt a request with exponential backoff and retry logic."""
    attempt = 0
    while attempt < retries:
        async with semaphore:
            try:
                async with session.post(url, headers=headers, json=json) as response:
                    if response.status in [200, 201]:
                        return await response.json()
                    elif response.status == 429:
                        retry_after = int(response.headers.get("Retry-After", 60))
                        logging.warning(f"Rate limit exceeded. Retrying after {retry_after} seconds.")
                    else:
                        raise RuntimeError(f"API returned a non-200 status code: {response.status}")
            except (ClientError, asyncio.TimeoutError) as e:
                logging.error(f"Request failed due to network error: {e}")
            # Wait before retrying with exponential backoff
            sleep_time = backoff_factor ** attempt
            logging.info(f"Retrying in {sleep_time} seconds...")
            await asyncio.sleep(sleep_time)
            attempt += 1
    # After all retries, raise an exception to indicate the request has ultimately failed
    raise RuntimeError("Request failed after multiple retries.")

async def generate_text_async_hf(session, text, prompt, generation_params, semaphore):
    payload = {
        "inputs": prompt.format(text=text),
        "parameters": {**generation_params}
    }
    # Call the request_with_retry function to handle potential retries
    response_json = await request_with_retry_hf(session, API_URL, HEADERS, payload, semaphore)
    generated_text = response_json[0].get("generated_text", "No text generated")
    if "error" in response_json:
        raise RuntimeError(f"API returned an error: {response_json['error']}")
    return generated_text




In [ ]:
async def run_batch(dataset, prompt, generation_params, api_provider, batch_size, sleep_time):
    results_lst = []
    semaphore = asyncio.BoundedSemaphore(128)
    timeout = ClientTimeout(total=60)

    #async with ClientSession(timeout=timeout) as session:
    async with ClientSession(timeout=timeout, connector=aiohttp.TCPConnector(ssl=ssl_context)) as session:
        for i in tqdm(range(0, len(dataset), batch_size), desc="Processing batches"):
            text_batch = dataset[i:i + batch_size]["sentence"]
            if api_provider == "HF":
                tasks = [generate_text_async_hf(session, text, prompt, generation_params, semaphore) for text in text_batch]
            elif api_provider == "OAI":
                tasks = [generate_text_async_oai(session, text, prompt, generation_params, semaphore) for text in text_batch]
            else:
                raise ValueError("Invalid API provider")
            results_batch = await asyncio.gather(*tasks)
            results_lst.extend(results_batch)
            await asyncio.sleep(sleep_time)

    return results_lst

In [ ]:
# run batch processing for chain-of-thought and self-consistency prompt
output_cot_multiple = []
for _ in range(SELF_CONSISTENCY_ITERATIONS):
    # run async function in jupyter notebook
    output_cot = await run_batch(dataset, prompt_ecb_uncertainty_cot_formatted, generation_params, API_PROVIDER, BATCH_SIZE, SLEEP_TIME)

    output_cot_multiple.append(output_cot)

print(output_cot[:3])

Processing batches: 100%|██████████| 10/10 [01:45<00:00, 10.55s/it]

[' {"reason": "The text states that evidence from a specific bank lending survey in January 2015 provided positive signals for a continued recovery of credit markets, and it concretely refers to a net easing of credit standards for all loan categories in a specific quarter. This information is based on observed data from the survey, making it a factual and definitive conclusion, and not indicating any doubt or ambiguity", "label": "certain"}', ' {"reason": "The text states a continuous downward trend in inflation outturns and subsequent downward revisions of the inflation outlook, all of which are observable facts. The target inflation rate of the Governing Council is also explicitly stated, making this sentence certain.", "label": "certain"}', ' {"reason": "The text uses the phrase \'could be considered\' which indicates a degree of speculation or possibility, and the concept of \'unanchoring of inflation expectations\' implies a potential scenario that is contingent on future events.

In [ ]:
# parse and clean outputs
random.seed(SEED)

labels = ["certain", "uncertain"]

# function to map each label string to a discrete category (for simple prompt)
def clean_output(string, random_choice=True):
    for category in labels:
        if category.lower() in string.lower():
            return category
    if random_choice:
        return random.choice(labels)  # random category if no category is found
    else:
        return "FAIL"

# function to parse chain-of-thought JSON output
def process_output_cot(output):
    try:
        output_dic = ast.literal_eval(output)
        return output_dic
    except Exception as e:
        # if json/dict parse fails, do simple search for occurance of first label term
        print(f"Parsing failed for output: {output}, Error: {e}")
        output_cl = clean_output(output, random_choice=True)
        output_dic = {"reason": "FAIL", "label": output_cl}
        return output_dic

# clean outputs for CoT + SC prompt
output_cot_multiple_cl = []
output_dic_lst = []
for i in range(SELF_CONSISTENCY_ITERATIONS):
    output_dic_step = [process_output_cot(output) for output in output_cot_multiple[i]]
    output_labels = [dic["label"] for dic in output_dic_step]
    output_cot_multiple_cl.append(output_labels)
    output_dic_lst.extend(output_dic_step)

Parsing failed for output:  {"reason": "The text includes the word 'could,' which indicates a possibility or potential scenario, and it also expresses doubt with the phrase 'a sign of an unanchoring of inflation expectations which required careful monitoring.' Additionally, it mentions 'possible shortcomings' in market-based indicators and market segments, which further contributes to the uncertainty. Therefore, the text should be labeled as 'uncertain'," "label": "uncertain"}, Error: invalid syntax (<unknown>, line 1)
Parsing failed for output:  {"reason": "The text presents a specific outcome (headline inflation being 1.6% in 2, Error: unterminated string literal (detected at line 1) (<unknown>, line 1)
Parsing failed for output:  {"reason": "The text discusses potential future scenarios for inflation dynamics, using the word 'could' and mentioning recovery in 2, Error: unterminated string literal (detected at line 1) (<unknown>, line 1)
Parsing failed for output:  {"reason": "The te

In [ ]:
# convert the CoT+SC output to dataframe for easier downstream processing
df_output = pd.DataFrame(data=output_cot_multiple_cl).T
df_output = df_output.rename(columns={0: "sc_iter1", 1: "sc_iter2", 2: "sc_iter3"})
df_output.head(3)

,sc_iter1,sc_iter2,sc_iter3
0,certain,certain,certain
1,certain,certain,certain
2,uncertain,uncertain,uncertain


In [ ]:
# find majority from multiple self-consistency runs
from collections import Counter
random.seed(SEED)

def find_majority(row):
    # Find majority
    count = Counter(row)
    majority = count.most_common(1)[0]
    # Check if it's a real majority or if all 3 labels appear 3 times
    if majority[1] > 1:
        return majority[0]
    else: # in case all 3 labels appear 3 times
        return random.choice(labels)


# majority for multiple self-consistency + CoT runs
df_output['label_llm_cot_multiple'] = df_output.apply(find_majority, axis=1)


df_output["text"] = dataset["sentence"]
df_output["date"] = dataset["date"]
df_output["expert_label"] = dataset["label"]
df_output["sampling_prob"] = dataset["sampling_prob"]
df_output["row_number"] = dataset["row_number"]

# also add the reasoning for the first iteration for inspection
df_output["reason_iter1"] = [dic["reason"] for dic in output_dic_lst[:len(df_output)]]
df_output["reason_iter2"] = [dic["reason"] for dic in output_dic_lst[len(df_output):len(df_output)*2]]
df_output["reason_iter3"] = [dic["reason"] for dic in output_dic_lst[len(df_output)*2:len(df_output)*3]]

df_output.head(3)

,sc_iter1,sc_iter2,sc_iter3,label_llm_cot_multiple,text,date,expert_label,sampling_prob,row_number,reason_iter1,reason_iter2,reason_iter3
0,certain,certain,certain,certain,Evidence from the January 2015 bank lending su...,19.02.2015,certain,0.049645,9918,The text states that the bank lending survey p...,The text describes the results of a bank lendi...,The text states that evidence from a specific ...
1,certain,certain,certain,certain,Inflation outturns had been on a continuous do...,19.02.2015,certain,0.049645,9929,The text states a continuous downward trend in...,The text states a continuous downward trend in...,The text states a continuous downward trend in...
2,uncertain,uncertain,uncertain,uncertain,It could therefore be considered that there wa...,19.02.2015,uncertain,0.049645,9902,The text includes the word 'could' which indic...,The text includes the word 'could' which indic...,The text uses the phrase 'could be considered'...


In [ ]:
# save for debugging and in case something fails after API calls
df_output.to_csv('surrogate_labels.csv', index=False)